# Figure 8: Glacier projections

In [ ]:
import numpy as np
import pandas as pd
import xarray as xr 
import geopandas as gpd
import shapely.geometry

import os
from glob import glob
from tqdm import tqdm
from tqdm.notebook import tqdm

import plotly.express as px
import plotly.graph_objects as go
from   plotly.subplots import make_subplots

import warnings
warnings.simplefilter("ignore")

In [ ]:
def postprocessing(ds, scenario): # clean dataframe
    ds = ds.to_dataframe()
    ds["scenario"] = scenario
    ds = ds.set_index("scenario", append=True)
    ds = ds.reorder_levels(['scenario', 'rgi_id', 'time'])
    return ds

## Data

### Time series

In [ ]:
basins = gpd.read_file("zip:////home/rooda/Dropbox/Patagonia/MS2 Results/zenodo/basins_boundaries.zip")
basins = basins[["basin_id", "basin_zone", "basin_name", "geometry"]].set_index("basin_id")
basins['basin_name']= basins['basin_name'].replace({'Santa Cruz': 'Santa Cruz                          '})
area = pd.read_csv("/home/rooda/Dropbox/Patagonia/MS2 Results/zenodo/dataset_historical.csv", index_col = "basin_id").area_RGI6
vol  = pd.read_csv("/home/rooda/Dropbox/Patagonia/MS2 Results/zenodo/dataset_historical.csv", index_col = "basin_id").vol_F19
basins = pd.concat([basins, area, vol], axis=1)

In [ ]:
# historical
ts_hist = xr.open_dataset("/home/rooda/OGGM_results/runs/OGGM_historical.nc")[["volume"]]
ids     = basins[basins.index.isin(ts_hist.rgi_id.to_pandas().tolist())]
ts_hist = ts_hist.assign_coords(rgi_id = ids.basin_zone.tolist())
ts_hist = ts_hist.groupby('rgi_id').sum() 
ts_hist_t = ts_hist.sum(dim = "rgi_id").assign_coords({"rgi_id": "total"}).expand_dims('rgi_id')
ts_hist   = xr.concat([ts_hist, ts_hist_t], dim = "rgi_id")

ts_hist_var  = postprocessing(ts_hist.std(dim="options"),  "historical")
ts_hist_mean = postprocessing(ts_hist.mean(dim="options"), "historical")

In [ ]:
# future
os.chdir('/home/rooda/OGGM_results/')
scenarios        = ["ct_random", "ssp126", "ssp245", "ssp370", "ssp585"]

ts_future_mean = []
ts_future_var  = []

for scenario in tqdm(scenarios): 
    ts_future_ssp   = xr.open_dataset("/home/rooda/OGGM_results/runs/OGGM_future_{}.nc".format(scenario))[["volume"]]
    ids             = basins[basins.index.isin(ts_future_ssp.rgi_id.to_pandas().tolist())]
    ts_future_ssp   = ts_future_ssp.assign_coords(rgi_id = ids.basin_zone.tolist())
    ts_future_ssp   = ts_future_ssp.groupby('rgi_id').sum()
    ts_future_ssp_t = ts_future_ssp.sum(dim = "rgi_id").assign_coords({"rgi_id": "total"}).expand_dims('rgi_id')
    ts_future_ssp   = xr.concat([ts_future_ssp, ts_future_ssp_t], dim = "rgi_id")
    
    ts_future_mean.append(postprocessing(ts_future_ssp.mean(dim="options"), scenario))
    ts_future_var.append(postprocessing(ts_future_ssp.std(dim="options"),   scenario))

ts_future_var  = pd.concat(ts_future_var)
ts_future_mean = pd.concat(ts_future_mean)

In [ ]:
# concat historical and future perios
ts_var  = pd.concat([ts_hist_var,  ts_future_var]).reset_index()
ts_mean = pd.concat([ts_hist_mean, ts_future_mean]).reset_index()

dict_replace = {"scenario": {"historical":'Historical', 
                             "ct_random":'Commitment run', 
                             "ssp126":'SSP 1-2.6', 
                             "ssp245":'SSP 2-4.5',
                             "ssp370":'SSP 3-7.0', 
                             "ssp585":'SSP 5-8.5'}}

ts_var      = ts_var.replace(dict_replace)
ts_mean     = ts_mean.replace(dict_replace)
ts_mean_ref = ts_mean[(ts_mean.time == 2020) & (ts_mean.scenario == "Historical")]

### Volume loss for each catchment

In [ ]:
volume_loss = xr.open_mfdataset("/home/rooda/OGGM_results/runs/OGGM_future_ssp*.nc").volume
volume_loss = volume_loss / volume_loss.sel(time = 2020)
volume_loss = volume_loss.sel(time = 2099)

basins  = pd.concat([basins, volume_loss.mean(dim = "options").to_dataframe().volume.rename("volume_mean")], axis=1)

## Plot elements

In [ ]:
# basemap for background
geo_map = gpd.read_file("/home/rooda/Dropbox/ArcGIS/Chile/south_america.shp")
geo_map = geo_map[(geo_map.CC == "CI") | (geo_map.CC == "AR")]
geo_map = geo_map.dissolve(by='REGION')
geo_map["geometry"] = geo_map.simplify(0.01)

poly_gdf = shapely.geometry.Polygon([(-76, -55.7), (-76, -40.52), (-68.05, -40.52), (-68.05, -55.7), (-76, -55.8)])
poly_gdf = gpd.GeoDataFrame([1], geometry=[poly_gdf], crs=geo_map.crs)

geo_map = geo_map.clip(poly_gdf)

In [ ]:
# hydrological zone divides
geo_lines = gpd.read_file("/home/rooda/Dropbox/Patagonia/GIS South/Basins_Patagonia_ice_divides.shp")

lats = []
lons = []

for feature in geo_lines.geometry:
    if isinstance(feature, shapely.geometry.linestring.LineString):
        linestrings = [feature]
    elif isinstance(feature, shapely.geometry.multilinestring.MultiLineString):
        linestrings = feature.geoms
    else:
        continue
    for linestring in linestrings:
        x, y = linestring.xy
        lats = np.append(lats, y)
        lons = np.append(lons, x)
        lats = np.append(lats, None)
        lons = np.append(lons, None)
        
lat_coords = [-43.2, -45.95,  -46.4,  -47.55,  -49.2,   -50.5,   -52.0, -53.1, -54.8]
lon_coords = [-71.2, -71.7,   -74.5,  -71.7,   -72.2,   -72.3,   -72.1, -71.7, -68.9]
names      = ["PPY", "PCA", "NPI-W", "NPI-E", "SPI-N", "SPI-C", "SPI-S", "GCN", "CDI"]
names  = ['<b>'+x+'</b>' for x in names]

## Plot

In [ ]:
cl = px.colors.colorbrewer.RdYlBu

scenarios   = ["Historical", "Commitment run", "SSP 1-2.6", "SSP 2-4.5", "SSP 3-7.0", "SSP 5-8.5"]
scen_colors = {"Historical":"rgba(0, 0, 0, 0.8)", 
               "Commitment run":"rgba(0, 0, 0, 0.5)", 
               "SSP 1-2.6":cl[9], 
               "SSP 2-4.5":cl[8], 
               "SSP 3-7.0":cl[3],
               "SSP 5-8.5":cl[1]}

shaded_colors = {"Historical":"rgba(0, 0, 0, 0.1)", 
                 "SSP 1-2.6": "rgba(49,54,149, 0.1)", 
                 "SSP 5-8.5": "rgba(215,48,39,0.1)"}

zone1  = ['PPY', 'PCA']
zone2  = ['NPI-E','NPI-W']
zone3  = ['SPI-N', 'SPI-C', 'SPI-S']
zone4  = ['GCN', 'CDI']

fig    = make_subplots(rows=3, cols=3, horizontal_spacing = 0.04, vertical_spacing = 0.05, 
                       shared_xaxes= True, shared_yaxes= True, subplot_titles = ["Ensemble volume loss in 2100", "Volume loss (relative to 2020)"],
                       specs=[[{"type": "scattergeo", "rowspan": 3},  {"type": "xy", "colspan": 2},           None],
                              [ None,                         {"type": "xy"},               {"type": "xy"}],
                              [ None,                         {"type": "xy"},               {"type": "xy"}]])

fig.add_trace(go.Choropleth(geojson = eval(geo_map['geometry'].to_json()),  locations = geo_map.index, z = geo_map['iso_num'], 
                            colorscale = ["#EAEAF2", "#EAEAF2"], showscale= False, marker_line_color ='white', marker_line_width=0.1), row=1, col=1)


colorbar_volume = dict(len=0.45, x=0.2, y= 0.77, title='Volume loss<br> (rel. to 2020)', ticksuffix = " %", thickness=20, tickwidth=1, title_font =  dict(size = 12))
fig.add_trace(go.Choropleth(geojson = eval(basins['geometry'].to_json()),  locations = basins.index, z = (basins['volume_mean']*100)-100, 
                            colorscale = [(0., cl[2]),(0.4, "#ffe9ba"),(1, cl[9])], marker_line_color ='white', marker_line_width=0.1, 
                            zmin = -100, zmax = -20, colorbar = colorbar_volume), row=1, col=1)
fig.add_annotation(text="(a)", font=dict(size=16), x=0.005, y=0.995,  xref = "paper", yref = "paper", showarrow=False)

## Add basin and hydrological zone names plus the hydro zone divides
fig.add_trace(go.Scattergeo(lon = lons, lat = lats, mode = 'lines', line = dict(width = 0.8, color = 'black'), opacity = 0.7, showlegend = False),row=1, col=1)  
fig.add_trace(go.Scattergeo(lon = lon_coords, lat=lat_coords, mode='text', text=names, textfont=dict(size=12, color = "rgba(0,0,0,0.7)"), showlegend = False),row=1, col=1)
fig.add_scattergeo(geojson = eval(basins['geometry'].to_json()), locations = basins.index, text = basins['basin_name'], mode = 'text', showlegend = False,
                   textfont=dict(size=11, color = "rgba(0,0,0,0.3)"),row=1, col=1)

fig.update_geos(showframe = True, framewidth = 1,  framecolor = "black", lonaxis_range=[-76, -68], lataxis_range=[-55.8, -40.5], 
                bgcolor = "rgb(255,255,255)", showland = False, showcoastlines = False, showlakes = False)

for t in range(0,6): # for each scenario 
    
    # total volume
    time_series_id    = ts_mean[ts_mean.rgi_id == "total"][ts_mean.scenario == scenarios[t]]
    time_series_sd_id = ts_var[ts_var.rgi_id == "total"][ts_var.scenario == scenarios[t]]    
    ts_ref_id         = ts_mean_ref[ts_mean_ref.rgi_id == "total"]
    
    fig.add_trace(go.Scatter(x=time_series_id.time, y=(time_series_id.volume/float(ts_ref_id.volume))-1, mode='lines', name= scenarios[t], 
                             line=dict(color=scen_colors[scenarios[t]], width = 1.5), showlegend=True), row=1, col=2)
    if t == 0 or t == 2 or t >= 5:
        fig.add_trace(go.Scatter(x=time_series_sd_id.time, y=((time_series_id.volume+time_series_sd_id.volume)/float(ts_ref_id.volume))-1, 
                                 line=dict(width=0), fillcolor=shaded_colors[scenarios[t]], showlegend=False, legendgroup='g1'), row=1, col=2)
        fig.add_trace(go.Scatter(x=time_series_sd_id.time, y=((time_series_id.volume-time_series_sd_id.volume)/float(ts_ref_id.volume))-1, 
                                 line=dict(width=0), fillcolor=shaded_colors[scenarios[t]], fill='tonexty', showlegend=False, legendgroup=t), row=1, col=2)
    fig.add_annotation(text="(b) Total", font = dict(size = 13), x = 1985, y = -0.92, showarrow=False, row=1, col=2)
    
    # zone 1
    time_series_id    = ts_mean[ts_mean.rgi_id.isin(zone1)][ts_mean.scenario == scenarios[t]].groupby("time").sum()
    time_series_sd_id = ts_var[ts_var.rgi_id.isin(zone1)][ts_var.scenario == scenarios[t]].groupby("time").sum()
    ts_ref_id         = ts_mean_ref[ts_mean_ref.rgi_id.isin(zone1)].groupby("time").sum()
    
    
    fig.add_trace(go.Scatter(x=time_series_id.index, y=(time_series_id.volume/float(ts_ref_id.volume))-1, mode='lines', name= scenarios[t], 
                             line=dict(color=scen_colors[scenarios[t]], width = 1.5), showlegend=False, legendgroup=t), row=2, col=2)
    if t == 0 or t == 2 or t >= 5:
        fig.add_trace(go.Scatter(x=time_series_sd_id.index, y=((time_series_id.volume+time_series_sd_id.volume)/float(ts_ref_id.volume))-1, 
                                 line=dict(width=0), fillcolor=shaded_colors[scenarios[t]], showlegend=False, legendgroup='g1'), row=2, col=2)
        fig.add_trace(go.Scatter(x=time_series_sd_id.index, y=((time_series_id.volume-time_series_sd_id.volume)/float(ts_ref_id.volume))-1, 
                                 line=dict(width=0), fillcolor=shaded_colors[scenarios[t]], fill='tonexty', showlegend=False, legendgroup=t), row=2, col=2)
    fig.add_annotation(text="(c) PPY + PCA", font = dict(size = 13), x = 2000, y = -0.92, showarrow=False, row=2, col=2)
    
    # zone 2
    time_series_id    = ts_mean[ts_mean.rgi_id.isin(zone2)][ts_mean.scenario == scenarios[t]].groupby("time").sum()
    time_series_sd_id = ts_var[ts_var.rgi_id.isin(zone2)][ts_var.scenario == scenarios[t]].groupby("time").sum()
    ts_ref_id         = ts_mean_ref[ts_mean_ref.rgi_id.isin(zone2)].groupby("time").sum()
    
    fig.add_trace(go.Scatter(x=time_series_id.index, y=(time_series_id.volume/float(ts_ref_id.volume))-1, mode='lines', name= scenarios[t], 
                             line=dict(color=scen_colors[scenarios[t]], width = 1.5), showlegend=False, legendgroup=t), row=2, col=3)
    if t == 0 or t == 2 or t >= 5:
        fig.add_trace(go.Scatter(x=time_series_sd_id.index, y=((time_series_id.volume+time_series_sd_id.volume)/float(ts_ref_id.volume))-1, 
                                 line=dict(width=0), fillcolor=shaded_colors[scenarios[t]], showlegend=False, legendgroup='g1'), row=2, col=3)
        fig.add_trace(go.Scatter(x=time_series_sd_id.index, y=((time_series_id.volume-time_series_sd_id.volume)/float(ts_ref_id.volume))-1, 
                                 line=dict(width=0), fillcolor=shaded_colors[scenarios[t]], fill='tonexty', showlegend=False, legendgroup=t), row=2, col=3)
    fig.add_annotation(text="(d) NPI", font = dict(size = 13), x = 1990, y = -0.92, showarrow=False, row=2, col=3)
    
    # zone 3
    time_series_id    = ts_mean[ts_mean.rgi_id.isin(zone3)][ts_mean.scenario == scenarios[t]].groupby("time").sum()
    time_series_sd_id = ts_var[ts_var.rgi_id.isin(zone3)][ts_var.scenario == scenarios[t]].groupby("time").sum()
    ts_ref_id         = ts_mean_ref[ts_mean_ref.rgi_id.isin(zone3)].groupby("time").sum()
    
    fig.add_trace(go.Scatter(x=time_series_id.index, y=(time_series_id.volume/float(ts_ref_id.volume))-1, mode='lines', name= scenarios[t], 
                             line=dict(color=scen_colors[scenarios[t]], width = 1.5), showlegend=False, legendgroup=t), row=3, col=2)
    if t == 0 or t == 2 or t >= 5:
        fig.add_trace(go.Scatter(x=time_series_sd_id.index, y=((time_series_id.volume+time_series_sd_id.volume)/float(ts_ref_id.volume))-1, 
                                 line=dict(width=0), fillcolor=shaded_colors[scenarios[t]], showlegend=False, legendgroup='g1'), row=3, col=2)
        fig.add_trace(go.Scatter(x=time_series_sd_id.index, y=((time_series_id.volume-time_series_sd_id.volume)/float(ts_ref_id.volume))-1, 
                                 line=dict(width=0), fillcolor=shaded_colors[scenarios[t]], fill='tonexty', showlegend=False, legendgroup=t), row=3, col=2)
    fig.add_annotation(text="(e) SPI", font = dict(size = 13), x = 1990, y = -0.92, showarrow=False, row=3, col=2)

    # zone 4
    time_series_id    = ts_mean[ts_mean.rgi_id.isin(zone4)][ts_mean.scenario == scenarios[t]].groupby("time").sum()
    time_series_sd_id = ts_var[ts_var.rgi_id.isin(zone4)][ts_var.scenario == scenarios[t]].groupby("time").sum()
    ts_ref_id         = ts_mean_ref[ts_mean_ref.rgi_id.isin(zone4)].groupby("time").sum()
    
    fig.add_trace(go.Scatter(x=time_series_id.index, y=(time_series_id.volume/float(ts_ref_id.volume))-1, mode='lines', name= scenarios[t], 
                             line=dict(color=scen_colors[scenarios[t]], width = 1.5), showlegend=False, legendgroup=t), row=3, col=3)
    if t == 0 or t == 2 or t >= 5:
        fig.add_trace(go.Scatter(x=time_series_sd_id.index, y=((time_series_id.volume+time_series_sd_id.volume)/float(ts_ref_id.volume))-1, 
                                 line=dict(width=0), fillcolor=shaded_colors[scenarios[t]], showlegend=False, legendgroup='g1'), row=3, col=3)
        fig.add_trace(go.Scatter(x=time_series_sd_id.index, y=((time_series_id.volume-time_series_sd_id.volume)/float(ts_ref_id.volume))-1, 
                                 line=dict(width=0), fillcolor=shaded_colors[scenarios[t]], fill='tonexty', showlegend=False, legendgroup=t), row=3, col=3)
    fig.add_annotation(text="(f) GCN + CDI", font = dict(size = 13), x = 1999, y = -0.92, showarrow=False, row=3, col=3)

fig.update_xaxes(griddash = "dot", gridcolor = "rgba(255,255,255,0.8)", zeroline=False, showline = True, linecolor = 'black', linewidth = 1, ticks="outside", mirror=True)
fig.update_yaxes(griddash = "dot", gridcolor = "rgba(255,255,255,0.8)", zeroline=False,  showline = True, linecolor = 'black', linewidth = 1, ticks="outside", mirror=True)    
    
fig.update_yaxes(tickformat = ',.0%', tickangle = -90, range = [-1, 0.19], dtick = 0.3)
fig.update_xaxes(dtick = 20)
fig.update_layout(legend=dict(yanchor="top", y=0.97, orientation = "h", xanchor="left", x=0.35, bgcolor = 'rgba(0,0,0,0.0)'))
fig.update_layout(width = 1000, height = 600, template = "seaborn", margin = dict(l=20, r=20, b=20, t=20), hovermode = False)

fig.write_image("/home/rooda/Dropbox/Patagonia/MS2 Results/Figure_8_Glacier_projection.png", scale=4)
fig.show()

## Text

In [ ]:
# loss more than 50% of its volume
volume_loss = xr.open_mfdataset("/home/rooda/OGGM_results/runs/OGGM_future_ssp*.nc").volume
volume_loss = volume_loss / volume_loss.sel(time = 2020)
volume_loss = volume_loss.sel(time = 2099)

renames = [y + "_" + str(x) for x, y in zip(np.tile(np.arange(1,481), 4),  volume_loss.options.values)]
volume_loss = volume_loss.assign_coords(options = renames)
volume_loss = volume_loss.to_dataframe()[["volume"]].reset_index()
volume_loss = volume_loss.pivot(index = "rgi_id", columns='options', values='volume')
volume_loss = pd.concat([basins, volume_loss], axis=1)

loss = []

for scenario in range(4, 1924):
    value = volume_loss[(volume_loss.iloc[:,scenario]*100)-100 < -50].area_RGI6.sum()*100 / basins.area_RGI6.sum()
    loss.append(value)

"Considering  the full set of SSP scenarios (n = 1920), {:.0f}% + {:.0f}% of the total glacier area will lose more than 50% of their current volume".format(
np.mean(loss),  np.std(loss))

In [ ]:
# committed loss
volume_loss = xr.open_dataset("/home/rooda/OGGM_results/runs/OGGM_future_ct_random.nc").volume
volume_loss = volume_loss.sum(dim = "rgi_id")
volume_loss = 1 - (volume_loss / volume_loss.sel(time = 2020))
volume_loss = volume_loss.sel(time = 2099) * 100

"{:.0f}% ± {:.0f}% of the total glacier ice is committed to melt in the long term".format(
    float(volume_loss.mean()), float(volume_loss.std()))

In [ ]:
# by scenario 
volume_loss = xr.open_mfdataset("/home/rooda/OGGM_results/runs/OGGM_future_ssp*.nc").volume
volume_loss = volume_loss.sum(dim = "rgi_id")
volume_loss = 1 - (volume_loss / volume_loss.sel(time = 2020))
volume_loss = volume_loss.sel(time = 2099) * 100

"Aggregating the time series by emission scenario (n = 480 per SSP), the volume loss varied from {:.0f} ± {:.0f}% in SSP1-2.6 to {:.0f} ± {:.0f}% in SSP5-8.5".format(
    float(volume_loss.sel(options = "ssp126").mean()),
    float(volume_loss.sel(options = "ssp126").std()),
    float(volume_loss.sel(options = "ssp585").mean()),
    float(volume_loss.sel(options = "ssp585").std()))

In [ ]:
# # by scenario and hydro zone
volume_loss = xr.open_dataset("/home/rooda/OGGM_results/runs/OGGM_future_ssp585.nc").volume
volume_loss = volume_loss.assign_coords(rgi_id = ids.basin_zone.tolist())
volume_loss = volume_loss.groupby('rgi_id').sum()

volume_loss_npi = volume_loss.sel(rgi_id = volume_loss.rgi_id.isin(["NPI-E", "NPI-W"])).sum(dim = "rgi_id")
volume_loss_npi = 1 - (volume_loss_npi / volume_loss_npi.sel(time = 2020))
volume_loss_npi = volume_loss_npi.sel(time = 2099) * 100

volume_loss_spi = volume_loss.sel(rgi_id = volume_loss.rgi_id.isin(["SPI-N", "SPI-C", "SPI-S"])).sum(dim = "rgi_id")
volume_loss_spi = 1 - (volume_loss_spi / volume_loss_spi.sel(time = 2020))
volume_loss_spi = volume_loss_spi.sel(time = 2099) * 100

volume_loss_cdi = volume_loss.sel(rgi_id = volume_loss.rgi_id.isin(["CDI", "GCN"])).sum(dim = "rgi_id")
volume_loss_cdi = 1 - (volume_loss_cdi / volume_loss_cdi.sel(time = 2020))
volume_loss_cdi = volume_loss_cdi.sel(time = 2099) * 100

"In NPI, SPI and the southern area (GCN and CDI), the percentage loss under SSP 5-8.5 will be {:.0f}% ± {:.0f}%, {:.0f}% ± {:.0f}% and {:.0f}% ± {:.0f}%, respectively".format(
    volume_loss_npi.mean(), volume_loss_npi.std(),
    volume_loss_spi.mean(), volume_loss_spi.std(),
    volume_loss_cdi.mean(), volume_loss_cdi.std())

## Review

In [ ]:
fig = px.scatter(y = basins.geometry.centroid.y, x = (basins['volume_mean']*100)-100, 
           width = 800, height = 800, size = np.sqrt(basins.vol_F19)*2, range_x = [-100,0], 
           labels={"x": "Volume loss in 2100 (rel. to 2020)",
                     "y": "Latitude (ºS)"  })
fig.update_layout(font_size = 16)
fig.write_image("/home/rooda/Dropbox/Patagonia/MS2 Results/Figure_8_Glacier_projection_review.png", scale=4)